In [34]:
#!huggingface-cli login
#!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [5]:
# !pip install folium==0.2.1
# !pip install folium==0.9.1
!sudo apt-get install git-lfs
! pip install -q transformers datasets
!pip install wandb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 65 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 2s (1377 kB/s)  
Selecting previously unselected package git-lfs.
(Reading database ... 108827 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
Unpacking git-lfs (2.9.2-1) ...
Setting up git-lfs (2.9.2-1) ...
Processing triggers for man-db (2.9.1-1) ...


In [7]:
! git clone https://github.com/csebuetnlp/xl-sum
! pip install --upgrade -r ./xl-sum/seq2seq/requirements.txt
! pip install --upgrade transformers/

# install rouge module and dependecies
! pip install -r ./xl-sum/multilingual_rouge_scoring/requirements.txt
! python -m unidic download # for japanese segmentation
! pip install --upgrade ./xl-sum/multilingual_rouge_scoring
! python -m nltk.downloader punkt

Cloning into 'xl-sum'...
remote: Enumerating objects: 1156, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 1156 (delta 19), reused 30 (delta 12), pack-reused 1109
Receiving objects: 100% (1156/1156), 5.53 MiB | 3.91 MiB/s, done.
Resolving deltas: 100% (309/309), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 4.0 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.4/408.4 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=89eb7d94534e75882234f27ee8027d30f4588433be3

In [10]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"]='false'

In [11]:
import datasets
import nltk
import numpy as np

import logging

logger = logging.getLogger(__name__)

## HF auth

In [12]:
!git config --global credential.helper store

## Model and tokenizer

In [13]:
# !pip install git+https://github.com/csebuetnlp/normalizer
!pip install sentencepiece
# !pip install transformers

In [14]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
CKPT = 'google/mt5-base'
from transformers import AutoTokenizer, T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained(CKPT)
model = T5ForConditionalGeneration.from_pretrained(CKPT)

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

## Dataset preparation

In [15]:
from datasets import concatenate_datasets, load_dataset

train_data = load_dataset("csebuetnlp/squad_bn", split="train")
valid_data = load_dataset("csebuetnlp/squad_bn", split="validation")
test_data = load_dataset("csebuetnlp/squad_bn", split="test")

concat_dataset = concatenate_datasets([train_data, valid_data, test_data])
dataset=concat_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset squad_bn downloaded and prepared to /root/.cache/huggingface/datasets/csebuetnlp___squad_bn/squad_bn/0.0.1/17a6d6abc976f299afda17ca9b5ce08a022ecafabe24b3362e16a3093c32df4b. Subsequent calls will reuse this data.


In [16]:
del concat_dataset, train_data, valid_data, test_data

In [17]:

small_dataset= dataset.filter(lambda example: len(example['answers']['text'])>0)
small_dataset= small_dataset.filter(lambda example: len(example['context'])>300)
# for i in range(0,10):
#     print(small_dataset['train'][i])

  0%|          | 0/124 [00:00<?, ?ba/s]

  0%|          | 0/72 [00:00<?, ?ba/s]

In [ ]:
# 0.2 * len(dataset)

In [19]:
split_data = small_dataset.train_test_split(test_size=0.2, seed= 42, shuffle=False)

In [20]:
dataset = split_data

In [21]:
del split_data

In [22]:
dataset['test'][13537]

{'id': 'bengali-6703107172676988032-5-4355',
 'title': 'পাল সাম্রাজ্য',
 'context': 'শশাঙ্কের রাজ্যের পতনের পর বাংলা অঞ্চলে নৈরাজ্য দেখা দেয়। এই সময় এই অঞ্চলে কোনও কেন্দ্রীয় শাসক ছিলেন না। ক্ষুদ্র গোষ্ঠীপতিরা নিরন্তর নিজেদের মধ্যে যুদ্ধে লিপ্ত ছিলেন। সমসাময়িক গ্রন্থে এই অবস্থাটিকে ‘মাৎস্যন্যায়’ (অর্থাৎ বড়ো মাছ যেমন ছোটো মাছকে খেয়ে ফেলে, সেই রকম অবস্থা) বলে বর্ণনা করা হয়েছে। এই সময়েই গোপাল প্রথম পাল রাজা হিসেবে সিংহাসনে আরোহণ করেন। খালিমপুর তাম্রলিপি থেকে অনুমিত হয়, বাংলা অঞ্চলের ‘প্রকৃতি’ (জনসাধারণ) তাঁকে রাজা নির্বাচিত করেছিল।[7] প্রায় ৮০০ বছর পরে তারানাথও লিখেছেন যে, বাংলার জনসাধারণ গণতান্ত্রিক পদ্ধতিতে তাঁকে নির্বাচিত করেছিল। যদিও এই ঘটনাটি কিংবদন্তির আকারে প্রচলিত এবং ঐতিহাসিকভাবে নির্ভরযোগ্য তথ্য নয়। এই কিংবদন্তি অনুসারে, নৈরাজ্যের এক যুগের পর জনসাধারণ পরপর একাধিক রাজাকে নির্বাচিত করেছিলেন। কিন্তু তাঁদের সকলকেই নির্বাচনের পরের রাতেই এক নাগ রানি ভক্ষণ করেন। গোপাল সেই নাগ রানিকে হত্যা করতে সমর্থ হন এবং সিংহাসনে আসীন থাকতে সমর্থ হন।[17] ঐতিহাসিক প্রমাণ নির্দেশ করে যে, গোপ

In [23]:
def format_dataset(example):
  
        return {'input': 'answer: ' + example['answers']['text'][0] + ' context: ' + example['context'], 'target': example['question']}

In [24]:
dataset = dataset.map(format_dataset, remove_columns=dataset['train'].column_names)


  0%|          | 0/54150 [00:00<?, ?ex/s]

  0%|          | 0/13538 [00:00<?, ?ex/s]

In [25]:
# from transformers import set_seed

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], truncation=True, padding='max_length', max_length=564)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], truncation=True, padding='max_length', max_length=64)

    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [26]:
dataset = dataset.map(convert_to_features, batched=True, remove_columns=dataset['train'].column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

dataset.set_format(type='torch', columns=columns)

  0%|          | 0/55 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

In [28]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

# # Metric

In [29]:
from rouge_score import rouge_scorer, scoring

def add_newline_to_end_of_each_sentence(x):
        return "\n".join(nltk.sent_tokenize(x))

def extract_rouge_mid_statistics(dct):
        new_dict = {}
        for k1, v1 in dct.items():
            mid = v1.mid
            new_dict[k1] = {stat: round(getattr(mid, stat), 4) for stat in ["precision", "recall", "fmeasure"]}
        return new_dict


def calculate_rouge(
        pred_lns,
        tgt_lns,
        use_stemmer=True,
        rouge_keys=["rouge1", "rouge2", "rougeL", "rougeLsum"],
        return_precision_and_recall=False,
        bootstrap_aggregation=True,
        newline_sep=True,
       rouge_lang='bengali'
    ):
        
        logger.info("Rouge lang: " + str(rouge_lang))
        scorer = rouge_scorer.RougeScorer(
            rouge_keys, lang=rouge_lang,
            use_stemmer=use_stemmer
        )
        aggregator = scoring.BootstrapAggregator()
        for pred, tgt in zip(tgt_lns, pred_lns):
            # rougeLsum expects "\n" separated sentences within a summary
            if newline_sep:
                pred = add_newline_to_end_of_each_sentence(pred)
                tgt = add_newline_to_end_of_each_sentence(tgt)
            scores = scorer.score(pred, tgt)
            aggregator.add_scores(scores)

        if bootstrap_aggregation:
            result = aggregator.aggregate()
            if return_precision_and_recall:
                return extract_rouge_mid_statistics(result)  # here we return dict
            else:
                results_precision = {k: round(v.mid.precision * 100, 4) for k, v in result.items()}
                results_recall = {k: round(v.mid.recall * 100, 4) for k, v in result.items()}
                results_fmeasure = {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}
            
            return {
                    "rouge1_precision": results_precision.get('rouge1'),
                    "rouge1_recall": results_recall.get('rouge1'),
                    "rouge1_fmeasure": results_fmeasure.get('rouge1'),

                    "rouge2_precision": results_precision.get('rouge2'),
                    "rouge2_recall": results_recall.get('rouge2'),
                    "rouge2_fmeasure": results_fmeasure.get('rouge2'),

                    "rougeL_precision": results_precision.get('rougeL'),
                    "rougeL_recall": results_recall.get('rougeL'),
                    "rougeL_fmeasure": results_fmeasure.get('rougeL'),
                
                    "rougeLsum_precision": results_precision.get('rougeLsum'),
                    "rougeLsum_recall": results_recall.get('rougeLsum'),
                    "rougeLsum_fmeasure": results_fmeasure.get('rougeLsum'),
                }
#                 return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}

        else:
            return aggregator._scores  # here we return defaultdict(list)

        
metric_fn = calculate_rouge


In [30]:
import numpy as np
from datasets import load_metric
# from sacrebleu.metrics import BLEU
# bleu = BLEU(max_ngram_order=4)
# bleu1 = BLEU(max_ngram_order=1)
# bleu2 = BLEU(max_ngram_order=2)
# bleu3 = BLEU(max_ngram_order=3)

from torchmetrics import SacreBLEUScore

bleu_metric1 = SacreBLEUScore(n_gram=1)
bleu_metric2 = SacreBLEUScore(n_gram=2)
bleu_metric3 = SacreBLEUScore(n_gram=3)
bleu_metric4 = SacreBLEUScore(n_gram=4)

# from nlgeval import NLGEval
# nlgeval = NLGEval()  # loads the models
# metrics_dict = nlgeval.compute_metrics(references, hypothesis)

# metric = load_metric("sacrebleu")
meteor = load_metric('meteor')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
        
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    
    decoded_preds_rouge = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels_rouge = tokenizer.batch_decode(labels, skip_special_tokens=True)
   # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result_rouge = metric_fn(decoded_preds_rouge, decoded_labels_rouge)
    
#     result_bleu = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     result_bleu4 = round(bleu.corpus_score(decoded_preds, decoded_labels).score, 4)
#     result_bleu1 = round(bleu1.corpus_score(decoded_preds, decoded_labels).score, 4)
#     result_bleu2 = round(bleu2.corpus_score(decoded_preds, decoded_labels).score, 4)
#     result_bleu3 = round(bleu3.corpus_score(decoded_preds, decoded_labels).score, 4)
    result_bleu1 = bleu_metric1(decoded_preds, decoded_labels)
    result_bleu2 = bleu_metric2(decoded_preds, decoded_labels)
    result_bleu3 = bleu_metric3(decoded_preds, decoded_labels)
    result_bleu4 = bleu_metric4(decoded_preds, decoded_labels)
    
    meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    
    result = result_rouge
#     result["bleu-1"] = result_bleu1
#     result["bleu-2"] = result_bleu2
#     result["bleu-3"] = result_bleu3
#     result["bleu-4"] = result_bleu4
    
    result["bleu-1"] = round((torch.FloatTensor(result_bleu1).item() * 100), 4)
    result["bleu-2"] = round((torch.FloatTensor(result_bleu2).item() * 100), 4)
    result["bleu-3"] = round((torch.FloatTensor(result_bleu3).item() * 100), 4)
    result["bleu-4"] = round((torch.FloatTensor(result_bleu4).item() * 100), 4)
#     result["sacrebleu"] =  round(result_bleu['score'], 4)

    
    result["meteor"] = round(meteor_result["meteor"],4)
#     metrics_dict = nlgeval.compute_metrics(decoded_preds, decoded_labels)
#     result['nlg'] = metrics_dict
    
#     result["gen_len"] = np.mean(prediction_lens)
#     result = {k: round(v,4) for k, v in result.items()}
    return result

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


### Set the training arguments

In [46]:
from transformers import set_seed
set_seed(42)
# set training arguments - Feel free to adapt it
training_args = Seq2SeqTrainingArguments(
    output_dir="squad-bn-mt5-base2",
    per_device_train_batch_size=4,
    num_train_epochs=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=500,
    save_strategy="epoch",
#     evaluation_strategy = "steps",
    #save_steps=1000,
    #eval_steps=1000,
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True,
    push_to_hub=True
    #fp16=True, 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [47]:
# from transformers import default_data_collator
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer
# data_collator = default_data_collator
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [48]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    # data_collator=T2TDataCollator(),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/Tahsin-Mayeesha/squad-bn-mt5-base2 into local empty directory.


In [49]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [50]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 54150
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 54152


Epoch,Training Loss,Validation Loss


RuntimeError: CUDA out of memory. Tried to allocate 734.00 MiB (GPU 0; 15.90 GiB total capacity; 13.52 GiB already allocated; 447.75 MiB free; 14.64 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.evaluate(eval_dataset=dataset['test'])

In [ ]:
trainer.save_model()

In [ ]:
# tokenizer.save_pretrained('/content/test2-qgen')
tokenizer.save_pretrained('./squad-bn-mt5-base')

In [ ]:
trainer.create_model_card()

In [ ]:
trainer.push_to_hub()

In [ ]:
#import shutil shutil.rmtree("./final-squad-bn-qgen-mt5-base-all-metric")

In [ ]:
tokenizer.push_to_hub("jannatul17/squad-bn-mt5-base")

In [51]:
 !pip freeze > '../working/requirement.txt'
